In [11]:
import csv
import utils

from collections import defaultdict
from copy import deepcopy

In [2]:
raw_data = []
with open("../data/MC ARC continuation - continuation-2.csv") as f:
    reader = csv.DictReader(f)
    for line in reader:
        raw_data.append(line)

In [3]:
idx_key_fixed = deepcopy(raw_data)
for i, entry in enumerate(idx_key_fixed):
    # fixes idx issue by setting it to the previous number if it is empty.
    if entry['idx'] == '':
        entry['idx'] = idx_key_fixed[i-1]['idx']
    
    # using the same logic as above, fixes the issue of mapping the values to vp1_cont and vp2_cont by using the previous key
    if entry['key'] == '':
        entry['key'] = idx_key_fixed[i-1]['key']

In [4]:
# save it all in the following format:
# {
#     1: {
#         'verb1': '',
#         'verb2': ''
#     }
# }

full_fixed = defaultdict(lambda: defaultdict(list))
for entry in idx_key_fixed:
    if entry['type'] == '':
        full_fixed[entry['idx']][entry['key']] = entry['value']
    else:
        # an old thing I wrote, ignore for now but keep around.
        # full_fixed[entry['idx']][entry['key']].append(entry['concated'].split("||"))
        full_fixed[entry['idx']][entry['key']].append((entry['value'], entry['type']))

In [10]:
# example:
full_fixed['1']

defaultdict(list,
            {'verb1': 'did',
             'verb2': 'does',
             'subj': 'The teacher',
             'vp1': 'met the Illinois governor at a Greek restaurant',
             'vp2': 'finds humor in the worst situations',
             'prn': 'he',
             'name1': 'Erika',
             'name2': 'Cameron',
             'vp1_cont': [('Which restaurant was it?', 'lexical overlap'),
              ("I didn't know the governor likes Greek food.",
               'lexical overlap'),
              ('Where was the restaurant located?', 'lexical overlap'),
              ('Was it only the governor, or were there other people as well?',
               'lexical overlap'),
              ('Is any of them Greek?', 'lexical overlap'),
              ('Did they take a picture together?', 'situational inference'),
              ('I wonder if they talked about educational policies.',
               'situational inference'),
              ('I thought the two were incompatible.',
   

In [6]:
# csv format

fixed_csv_format = []
for idx, entry in full_fixed.items():
    assert len(entry['vp1_cont']) == len(entry['vp2_cont'])
    
    # ignore unfilled cases
    if entry['vp1_cont'][0][0] == '':
        break

    i = 1
    for continuation, cont_type in entry['vp1_cont']:
        
        instance = {'idx': idx}
        for k, v in entry.items():
            if k not in ['vp1_cont', 'vp2_cont']:
                instance[k] = v
        
        instance['continuation'] = continuation
        instance['continuation_type'] = cont_type
        instance['vp'] = '1'
        instance['continuation_id'] = i
        fixed_csv_format.append(instance)
        i+=1
        
    i = 1
    for continuation, cont_type in entry['vp2_cont']:

        instance = {'idx': idx}
        for k, v in entry.items():
            if k not in ['vp1_cont', 'vp2_cont']:
                instance[k] = v
                
        instance['continuation'] = continuation
        instance['continuation_type'] = cont_type
        instance['vp'] = '2'
        instance['continuation_id'] = i
        fixed_csv_format.append(instance)
        i+=1

In [7]:
# len(fixed_csv_format)

# check if we have assigned ids in a manner that each instance is uniquely identifiable.

ids = []
for entry in fixed_csv_format:
    ids.append(f"{entry['idx']}_{entry['vp']}_{entry['continuation_id']}")
    
len(set(ids)) == len(fixed_csv_format)

True

In [8]:
len(fixed_csv_format)

900

In [12]:
# write to csv
utils.write_dict_list_to_csv(fixed_csv_format, "../data/manual-items.csv")